In [ ]:
!pip install thop
!pip install onnxruntime
#!pip install onnxruntime-gpu
!pip install ultralytics

In [33]:
import gc
import os
import psutil
import shutil
import time
import cv2
import numpy as np
import torch
import onnxruntime as ort
from google.colab import drive
from copy import deepcopy
from ultralytics import YOLO
from thop import profile


In [3]:
drive.mount('/content/gdrive/')

!unzip -q /content/gdrive/MyDrive/data.zip

Mounted at /content/gdrive/


In [31]:
def get_memory_usage():
    """Получение текущего потребления памяти"""
    process = psutil.Process()
    return process.memory_info().rss / 1024 / 1024  # в МБ

def run_inference_pytorch(model, images, device='cpu'):
    """Запуск инференса на PyTorch модели"""
    model.model.to(device)
    times = []

    for img_name in images:
        img_path = os.path.join(test_folder, img_name)
        start_time = time.time()

        results = model(img_path, verbose=False)

        end_time = time.time()
        times.append(end_time - start_time)

    return times

def run_inference_onnx(model_path, images, device='cpu'):
    """Запуск инференса на ONNX модели"""
    providers = ['CPUExecutionProvider'] if device == 'cpu' else ['CUDAExecutionProvider']
    session = ort.InferenceSession(model_path, providers=providers)
    times = []

    for img_name in images:
        img_path = os.path.join(test_folder, img_name)

        # Подготовка входных данных для ONNX
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (640, 640))
        img = img.astype(np.float32) / 255.0
        img = np.transpose(img, (2, 0, 1))
        img = np.expand_dims(img, axis=0)

        input_name = session.get_inputs()[0].name

        start_time = time.time()

        outputs = session.run(None, {input_name: img})

        end_time = time.time()
        times.append(end_time - start_time)

    return times


In [13]:
def save_to_colab(model, images, output_dir):
    """Сохранение изображений с bounding boxes в Google Colab"""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for img_name in images:
        img_path = os.path.join(test_folder, img_name)
        results = model(img_path, verbose=False)

        # Сохранение результата
        for r in results:
            im_array = r.plot()  # plot a BGR numpy array of predictions
            im = cv2.cvtColor(im_array, cv2.COLOR_BGR2RGB)
            output_path = os.path.join(output_dir, img_name)
            cv2.imwrite(output_path, im_array)

    print(f"Аннотированные изображения сохранены в {output_dir}")

def save_to_drive(colab_output_folder, drive_output_folder):
    """Сохранение изображений с bounding boxes в Google Drive"""

    os.makedirs(drive_output_folder, exist_ok=True)

    files_to_copy = [f for f in os.listdir(colab_output_folder)]

    for file_name in files_to_copy:
        file_path = os.path.join(colab_output_folder, file_name)
        if os.path.exists(file_path):
            shutil.copy(file_path, drive_output_folder)

    print(f"Аннотированные изображения сохранены в: {drive_output_folder}")


In [8]:
# Пути

test_folder = './test'
colab_output_folder = './test_results'
drive_output_folder = '/content/gdrive/MyDrive/test_results'

In [9]:
test_images = [f for f in os.listdir(test_folder) if f.lower().endswith('.jpg')]
test_count = len(test_images)

print(f"Количество фото в папке test: {test_count}")

Количество фото в папке test: 36


#### Сохранение фото с bounding boxes

In [10]:
# Загрузка моделей

pytorch_model = YOLO('08_best_model.pt')
onnx_model = '08_best_model.onnx'

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Используемое устройство: {device}")

Используемое устройство: cpu


In [11]:
save_to_colab(pytorch_model, test_images, colab_output_folder)
save_to_drive(colab_output_folder, drive_output_folder)

Аннотированные изображения сохранены в ./test_results
Аннотированные изображения сохранены в: /content/gdrive/MyDrive/test_results


#### Измерения на CPU

In [27]:
# PyTorch

# Потребление памяти перед инференсом
initial_memory = get_memory_usage()
gc.collect()
start_time = time.time()

pytorch_cpu_times = run_inference_pytorch(pytorch_model, test_images, 'cpu')
pytorch_cpu_total_time = time.time() - start_time

# Потребление памяти после инференса
final_memory = get_memory_usage()
memory_usage = final_memory - initial_memory

print(f"PyTorch - Общее время обработки: {pytorch_cpu_total_time:.2f} сек")
print(f"PyTorch - Среднее время на снимок: {np.mean(pytorch_cpu_times):.4f} сек")
print(f"PyTorch - Потребление памяти: {memory_usage:.2f} МБ")


PyTorch - Общее время обработки: 50.13 сек
PyTorch - Среднее время на снимок: 1.3924 сек
PyTorch - Потребление памяти: -0.07 МБ


In [29]:
# ONNX

# Потребление памяти перед инференсом
initial_memory = get_memory_usage()
gc.collect()
start_time = time.time()
onnx_cpu_times = run_inference_onnx(onnx_model, test_images, 'cpu')
onnx_cpu_total_time = sum(onnx_cpu_times)

# Потребление памяти после инференса
final_memory = get_memory_usage()
memory_usage = final_memory - initial_memory

print(f"ONNX - Общее время обработки: {onnx_cpu_total_time:.2f} сек")
print(f"ONNX - Среднее время на снимок: {np.mean(onnx_cpu_times):.4f} сек")
print(f"ONNX - Потребление памяти: {memory_usage:.2f} МБ")

ONNX - Общее время обработки: 49.26 сек
ONNX - Среднее время на снимок: 1.3684 сек
ONNX - Потребление памяти: -4.46 МБ
